# PySpark Text Classification
#### MultiClass Text Classification
#### Task
+ predict the subject category given a course title or text

In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext(master="local[2]")

In [3]:
# Launch UI
sc

<SparkContext master=local[2] appName=pyspark-shell>

In [4]:
# Create a spark session
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("TextClassifierwithPySpark").getOrCreate()